In [201]:
import glob
import re
import pandas as pd
from yattag import Doc
import unicodedata

In [202]:
doc, tag, text = Doc().tagtext()

In [203]:
txt_files = glob.glob('data/**/**/*.txt')

In [204]:
# AVISO DE ADJUDICACAO E HOMOLOGACAO
all_contracts = []
for txt in txt_files:
    instances = re.findall('AVISO (?:de)? ADJUDICACAO (?:.*HOMOLOGACAO)([\s\S]*?)\.\n',
                            unicodedata.normalize('NFKD', open(txt, encoding='utf-8').read()), 
                            re.IGNORECASE)
    all_contracts.extend(instances)

In [205]:
print(all_contracts)

deiras\nplasticas, para atender demandas da Secretaria de Estado de Seguranca Publica do Distrito Federal. A\nSSPDF informa, com fulcro no inciso IX do artigo 17 e inciso VI do artigo 13 do Decreto no\n10.024/2019, a ADJUDICACAO do item 1 a Aiala Eventos Eireli, CNPJ 10.862.668/0001-00, no\nvalor total de R$ 7.740,00; do item 2 a Supreme Artigos de Plastico Ltda, CNPJ 23.036.879/0001-\n27, no valor total de R$3.229,20. A SSPDF informa ainda a HOMOLOGACAO do certame com\nfulcro no inciso VI do Artigo 13, do Decreto no 10.024/2019', '\nPREGAO ELETRONICO No 30/2019\nProcesso: 00050.00036456/2019-40. Tipo: Menor Preco. Objeto: Registro de Precos para futura aquisicao\nde Rocadeira manual com motor e Podador de galhos com motor, para atender demandas da Secretaria de\nEstado de Seguranca Publica do Distrito Federal. A SSPDF informa, com fulcro no inciso IX do artigo 17\ndo Decreto no 10.024/2019, a ADJUDICACAO do item 1 a Comercial Vanguardeira Eireli, CNPJ\n10.942.831/0001-36, no valor uni

In [206]:
regex_dict = {
  'processo': 'PROCESSO[\s\S]*(\d{5}[\.|\-]\d{8}/\d{4}-\d{2})',
  'tipo': 'TIPO[:]?([\s\S]*?)\.',
  'objeto': 'OBJETO[\s\S]*?:?([\s\S]*?)\.',
  # 'licitante_vencedor': '',
  'CNPJ': 'CNPJ[\s\S]*(\d{2}.\d{3}.\d{3}/\d{4}-\d{2})',
  'valor': 'no valor[\s\S]*?(\d+(?:\.\d+)*,\d\d)',
  'valor_total': '[total licitado|totalizando](\d+(?:\.\d+)*,\d\d)',
  'modo_disputa': 'Modo de disputa[:]?([\s\S]*?)\.',              
}

df_dict = {
    'processo': [],
    'tipo': [],
    'objeto': [],
    # 'licitante_vencedor': [],
    'CNPJ': [],
    'valor': [],
    'valor_total': [],
    'modo_disputa': [],
    'texto': [],   
}

In [207]:
for contract in all_contracts:
    df_dict['texto'].append(contract)
    for field in regex_dict:
        match = re.search(regex_dict[field], contract, re.IGNORECASE)
        if match:
            res = tuple(x for x in match.groups() if x is not None)
            df_dict[field].append(res[0])
        else:
            df_dict[field].append(np.nan)

df = pd.DataFrame.from_dict(df_dict)
df.to_csv('adjudicacao_regex.csv')

In [209]:
dados_adjudicacao = pd.read_csv('adjudicacao_regex.csv')
dados_adjudicacao

,Unnamed: 0,processo,tipo,objeto,CNPJ,valor,valor_total,modo_disputa,texto
0,0,00097-00003259/2018-27,Menor Preco,NaN,08.012.469/0001-52,"121.500,00",NaN,NaN,"\nPREGAO ELETRONICO No 21/2018-METRO-DF,\nOCOR..."
1,1,NaN,Menor Preco,NaN,92.863.505/0001-06,"210.540,00",NaN,NaN,"\nPREGAO ELETRONICO No 23/2018-METRO-DF, OCORR..."
2,2,NaN,Menor Preco,NaN,51.832.889/0001-57,"200.100,00",NaN,NaN,"\nPREGAO ELETRONICO No 17/2018-METRO-DF,\nOCOR..."
3,3,00097-00001909/2018-08,Menor Preco,NaN,72.649.361/0001-74,"42.023,90",NaN,NaN,"\nPREGAO ELETRONICO No 27/2018-METRO-DF,\nOCOR..."
4,4,00097-00006864/2018-50,Menor Preco,NaN,10.336.598/0001-48,"7.468,47",NaN,NaN,"\nPREGAO ELETRONICO No 24/2018-METRO-DF,\nOCOR..."
...,...,...,...,...,...,...,...,...,...
106,106,00053-00094716/2019-17,Menor preco,Contratacao \nde empresa especializada para c...,11.064.583/0001-30,NaN,"930.000,00",NaN,\nRDC No 05/2020\nProcesso: 00053-00094716/201...
107,107,00050-00008415/2019-63,Menor Preco,Registro de precos \npara futura aquisicao de...,36.325.157/0001-34,"2,94","2,94",NaN,\nPREGAO ELETRONICO No 02/2020\nProcesso: 0005...
108,108,00050-00063790/2019-76,Menor Preco,Contratacao de empresa especializada para pre...,00.611.418/0001-35,"26.970,00","26.970,00",Aberto,\nPREGAO ELETRONICO No 04/2020\nProcesso: 0005...
109,109,00050-00035077/2019-32,Menor Preco,Registro de Precos \npara futura aquisicao de...,28.128.604/0001-37,"187,50","187,50",NaN,SRP\nPREGAO ELETRONICO No 20/2020\nProcesso:...


In [210]:
dados_adjudicacao.isnull().sum()

Unnamed: 0        0
processo         28
tipo             29
objeto           30
CNPJ             20
valor            44
valor_total      32
modo_disputa    109
texto             0
dtype: int64